# Predicting Temperature with RNN

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

In [2]:
#this version of numpy is required to avoid an error related to numpy defaulting to not allowing pickle files
!pip install numpy==1.16.2

In [3]:
!pip install pandas==0.24

In [0]:
import tensorflow as tf
from tensorflow.keras import datasets, preprocessing, models, layers
import numpy as np
import pandas as pd

## Connect to Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
df = pd.read_csv('/content/gdrive/My Drive/temperature/jena_climate_2009_2016.csv')

## Work with Temperature Data Only

In [0]:
df = df[['Date Time', 'T (degC)']]
df['DateTime'] = pd.to_datetime(df['Date Time'], format="%d.%m.%Y %H:%M:%S")

In [40]:
df.set_index(['DateTime'])

,Date Time,T (degC)
DateTime,,
2009-01-01 00:10:00,01.01.2009 00:10:00,-8.02
2009-01-01 00:20:00,01.01.2009 00:20:00,-8.41
2009-01-01 00:30:00,01.01.2009 00:30:00,-8.51
2009-01-01 00:40:00,01.01.2009 00:40:00,-8.31
2009-01-01 00:50:00,01.01.2009 00:50:00,-8.27
2009-01-01 01:00:00,01.01.2009 01:00:00,-8.05
2009-01-01 01:10:00,01.01.2009 01:10:00,-7.62
2009-01-01 01:20:00,01.01.2009 01:20:00,-7.62
2009-01-01 01:30:00,01.01.2009 01:30:00,-7.91


## Extract Time Series Data

In [0]:
time_series_df = df.set_index(['DateTime'])

In [42]:
time_series_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 420551 entries, 2009-01-01 00:10:00 to 2017-01-01 00:00:00
Data columns (total 2 columns):
Date Time    420551 non-null object
T (degC)     420551 non-null float64
dtypes: float64(1), object(1)
memory usage: 9.6+ MB


In [0]:
time_series_df.drop(['Date Time'], axis=1, inplace=True)

## Resample to Mean Daily Temperature

In [44]:
mean_daily_temp = time_series_df.resample('D').mean()
mean_daily_temp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2923 entries, 2009-01-01 to 2017-01-01
Freq: D
Data columns (total 1 columns):
T (degC)    2921 non-null float64
dtypes: float64(1)
memory usage: 45.7 KB


In [0]:
test_daily_temp = mean_daily_temp['2016']
train_daily_temp = mean_daily_temp['2009':'2015']

## Reset Indices

In [46]:
test_daily_temp.reset_index(inplace=True)
test_daily_temp.reset_index(inplace=True)
test_daily_temp.head()

,index,DateTime,T (degC)
0,0,2016-01-01,0.249236
1,1,2016-01-02,-0.975486
2,2,2016-01-03,-1.578056
3,3,2016-01-04,-2.081111
4,4,2016-01-05,-1.761319


In [47]:
train_daily_temp.reset_index(inplace=True)
train_daily_temp.reset_index(inplace=True)
train_daily_temp.head()

,index,DateTime,T (degC)
0,0,2009-01-01,-6.810629
1,1,2009-01-02,-3.728194
2,2,2009-01-03,-5.271736
3,3,2009-01-04,-1.375208
4,4,2009-01-05,-4.867153


## Extract Time Steps

In [0]:
train_x = train_daily_temp[['index']].to_numpy()
train_y = train_daily_temp[['T (degC)']].to_numpy()

In [52]:
print(train_x.shape)

(2556, 1)


In [0]:
test_x = test_daily_temp[['index']].to_numpy()
test_y = test_daily_temp[['T (degC)']].to_numpy()

## Create Generator

In [0]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [0]:
N_INPUT = 128
BATCH_SIZE = 64

In [0]:
train_gen = TimeseriesGenerator(train_x, train_y, length=N_INPUT, sampling_rate=7, batch_size=BATCH_SIZE)

In [82]:
print(len(train_gen))

38


In [83]:
batch_0 = train_gen[0]
x, y = batch_0
print(y.shape)

(64, 1)


In [93]:
print(x.shape)

(64, 19, 1)


In [0]:
test_gen = TimeseriesGenerator(test_x, test_y, length=N_INPUT, sampling_rate=7, batch_size=BATCH_SIZE)

## Define Dense Model

In [98]:
model = models.Sequential([
    layers.Dense(100, input_shape=(x.shape[1], x.shape[2]), activation='relu'),
    layers.Flatten(),
    layers.Dense(1)
])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 19, 100)           200       
_________________________________________________________________
flatten (Flatten)            (None, 1900)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 1901      
Total params: 2,101
Trainable params: 2,101
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.compile(optimizer='adam', loss='mse')

In [104]:
model.fit_generator(train_gen, epochs=5)

Epoch 1/5
38/38 [==============================] - 1s 14ms/step - loss: 7673.4290
Epoch 2/5
38/38 [==============================] - 0s 5ms/step - loss: 315.2329
Epoch 3/5
38/38 [==============================] - 0s 5ms/step - loss: 161.8855
Epoch 4/5
38/38 [==============================] - 0s 8ms/step - loss: 107.8195
Epoch 5/5
38/38 [==============================] - 0s 6ms/step - loss: 88.0511


In [0]:
example = test_gen[0]
x, y = example
y_pred = model.predict(x)

In [106]:
print(len(y_pred))

64


In [107]:
print(y_pred[0])

[2.9026558]


In [108]:
print(y[0])

[17.23909722]


## Define RNN Model

In [143]:
rnn_model = models.Sequential([
    layers.SimpleRNN(100, input_shape=(x.shape[1], x.shape[2]), return_sequences=True, dropout=0.5, recurrent_dropout=0.5),
    layers.SimpleRNN(200, dropout=0.5, recurrent_dropout=0.5),
    layers.Dense(1)
])
print(rnn_model.summary())

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_26 (SimpleRNN)    (None, 19, 100)           10200     
_________________________________________________________________
simple_rnn_27 (SimpleRNN)    (None, 200)               60200     
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 201       
Total params: 70,601
Trainable params: 70,601
Non-trainable params: 0
_________________________________________________________________
None


In [146]:
rnn_model.compile(optimizer='adam', loss='mse')
rnn_model.fit_generator(train_gen, epochs=5)

Epoch 1/5
38/38 [==============================] - 4s 113ms/step - loss: 63.4618
Epoch 2/5
38/38 [==============================] - 2s 55ms/step - loss: 61.8683
Epoch 3/5
38/38 [==============================] - 2s 53ms/step - loss: 62.6388
Epoch 4/5
38/38 [==============================] - 2s 57ms/step - loss: 63.3157
Epoch 5/5
38/38 [==============================] - 2s 55ms/step - loss: 61.4936


In [147]:
y_pred = rnn_model.predict(x)
print(y_pred[0])

[9.817606]


## Define LSTM Model

In [158]:
lstm_model = models.Sequential([
    layers.Bidirectional(layers.LSTM(100, return_sequences=True, recurrent_dropout=0.5), input_shape=(x.shape[1], x.shape[2])),
    layers.Bidirectional(layers.LSTM(100, recurrent_dropout=0.5)),
    layers.Dense(1)
])
print(lstm_model.summary())

W0523 01:17:51.801743 140148563203968 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f7533bf3da0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0523 01:17:51.811113 140148563203968 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f7533c0f198>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0523 01:17:51.817688 140148563203968 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f752bb676d8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0523 01:17:51.820705 140148563203968 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f752bb67e10>: Note that this layer is not optimized for performance. Please use tf.keras.laye

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_13 (Bidirectio (None, 19, 200)           81600     
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 19, 400)           641600    
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 200)               400800    
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 201       
Total params: 1,124,201
Trainable params: 1,124,201
Non-trainable params: 0
_________________________________________________________________
None


In [159]:
rnn_model.compile(optimizer='adam', loss='mse')
rnn_model.fit_generator(train_gen, epochs=5)

Epoch 1/5
38/38 [==============================] - 5s 133ms/step - loss: 61.7254
Epoch 2/5
38/38 [==============================] - 2s 57ms/step - loss: 61.1971
Epoch 3/5
38/38 [==============================] - 2s 53ms/step - loss: 61.6255
Epoch 4/5
38/38 [==============================] - 2s 55ms/step - loss: 61.4388
Epoch 5/5
38/38 [==============================] - 2s 54ms/step - loss: 61.4739


In [160]:
y_pred = rnn_model.predict(x)
print(y_pred[0])

[9.926286]
